In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))

%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np

from holoviews.streams import Pipe, Buffer
from scipy.ndimage.filters import uniform_filter1d
from tqdm import tnrange

import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from chipwhisperer.common.traces import Trace

def print_ec_conf():
    print(f"cw reports running:\t\t {scope.EC.check_status()}")
    print(f"cw reports edge_num:\t\t {scope.EC.edge_num}")
    print(f"cw reports settling_time:\t {scope.EC.settling_time}")
    print(f"cw reports pretrigger_ctr:\t {scope.EC.pretrigger_ctr}")
    print(f"cw reports edge_type:\t\t {scope.EC.edge_type}")
    print(f"cw reports threshold:\t\t {scope.EC.threshold}")
    
def main():
    scope.EC.threshold = 0.016
    scope.EC.edge_type = "falling_edge"
    scope.EC.settling_time = 128
    scope.EC.pretrigger_ctr = 6
    scope.EC.edge_num = 1
    scope.EC.start()
    print_ec_conf()
    
main()